In [6]:
!pip install cmake 

In [9]:
!pip install dlib --verbose

Using pip 25.0.1 from C:\Users\udayk\AppData\Roaming\Python\Python312\site-packages\pip (python 3.12)


In [10]:
!pip install numpy==1.26.4 opencv-python==4.11.0.86 pillow==10.3.0 ipywidgets==7.8.1 
!pip install face_recognition==1.3.0 

In [11]:
!pip install git+https://github.com/ageitgey/face_recognition_models 

  Cloning https://github.com/ageitgey/face_recognition_models to c:\users\udayk\appdata\local\temp\pip-req-build-c3vgz3b8
  Resolved https://github.com/ageitgey/face_recognition_models to commit e67de717267507d1e9246de95692eb8be736ab61
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/ageitgey/face_recognition_models 'C:\Users\udayk\AppData\Local\Temp\pip-req-build-c3vgz3b8'


In [6]:
!pip install scikit-learn

In [8]:
!pip install tensorflow


In [7]:
import face_recognition
import cv2
import numpy as np
import PIL
import ipywidgets
print("All imports successful!")

All imports successful!


In [1]:
import cv2
import numpy as np
import os
from collections import defaultdict
import io
from PIL import Image
import zipfile
from pathlib import Path
import uuid
import time
from IPython.display import display, HTML, FileLink, clear_output
import base64
import ipywidgets as widgets
from ipywidgets import Text, Button, Output, HBox, VBox, FloatProgress
import face_recognition

# Updated UI HTML/CSS with 3-column grid, larger cells, sequential face numbers, and no share options
after_ui_html = """
<style>
    * {{
        margin: 0;
        padding: 0;
        box-sizing: border-box;
        font-family: 'Roboto', -apple-system, BlinkMacSystemFont, 'Segoe UI', sans-serif;
    }}
    .container {{
        background: #ffffff;
        border-radius: 12px;
        box-shadow: 0 4px 20px rgba(0, 0, 0, 0.08);
        padding: 30px;
        width: 100%;
        max-width: 1200px;
        color: #202124;
        margin: 30px auto;
        border: 1px solid #dadce0;
    }}
    h1 {{
        text-align: center;
        margin-bottom: 25px;
        color: #202124;
        font-size: 28px;
        font-weight: 500;
        letter-spacing: -0.3px;
    }}
    #output {{
        background: #f1f3f4;
        border: 1px solid #dadce0;
        border-radius: 8px;
        padding: 20px;
        min-height: 200px;
        font-size: 14px;
        line-height: 1.6;
        white-space: pre-wrap;
        color: #5f6368;
        overflow-y: auto;
        box-shadow: inset 0 1px 3px rgba(0, 0, 0, 0.03);
    }}
    a {{
        color: #1a73e8;
        text-decoration: none;
        font-weight: 500;
        transition: color 0.2s ease;
    }}
    a:hover {{
        color: #1557b0;
        text-decoration: underline;
    }}
    .face-group {{
        margin-bottom: 30px;
    }}
    .face-label {{
        font-size: 18px;
        font-weight: 500;
        color: #202124;
        margin-bottom: 10px;
        background: #e8f0fe;
        padding: 8px 12px;
        border-radius: 4px;
        display: inline-block;
    }}
    .image-grid {{
        display: grid;
        grid-template-columns: repeat(3, 1fr);
        gap: 12px;
    }}
    .image-card {{
        position: relative;
        width: 100%;
        aspect-ratio: 1 / 1;
        min-height: 250px;
        overflow: hidden;
        border-radius: 4px;
        cursor: pointer;
    }}
    .image-card img {{
        width: 100%;
        height: 100%;
        object-fit: cover;
        border-radius: 4px;
        display: block;
        transition: filter 0.2s ease;
    }}
    .image-overlay {{
        position: absolute;
        top: 0;
        left: 0;
        right: 0;
        bottom: 0;
        background: rgba(0, 0, 0, 0.4);
        opacity: 0;
        border-radius: 4px;
        display: flex;
        flex-direction: column;
        justify-content: space-between;
        padding: 8px;
        transition: opacity 0.2s ease;
    }}
    .image-card:hover .image-overlay {{
        opacity: 1;
    }}
    .image-card:hover img {{
        filter: brightness(80%);
    }}
    .group-label {{
        font-size: 13px;
        font-weight: 500;
        color: #ffffff;
        background: rgba(0, 0, 0, 0.6);
        padding: 4px 8px;
        border-radius: 12px;
        display: inline-block;
        align-self: flex-start;
    }}
    .image-filename {{
        font-size: 12px;
        color: #ffffff;
        text-align: left;
        word-break: break-all;
        background: rgba(0, 0, 0, 0.6);
        padding: 4px 8px;
        border-radius: 4px;
    }}
    .modal {{
        display: none;
        position: fixed;
        top: 0;
        left: 0;
        width: 100%;
        height: 100%;
        background: rgba(0, 0, 0, 0.9);
        z-index: 1000;
        justify-content: center;
        align-items: center;
    }}
    .modal-content {{
        max-width: 90%;
        max-height: 90%;
        object-fit: contain;
    }}
    .close-btn {{
        position: absolute;
        top: 20px;
        right: 20px;
        color: #ffffff;
        font-size: 30px;
        font-weight: bold;
        cursor: pointer;
        background: none;
        border: none;
    }}
</style>
<div class="container">
    <h1>Face Grouping Results</h1>
    <div id="output">{}</div>
    <div id="modal" class="modal">
        <button class="close-btn" onclick="document.getElementById('modal').style.display='none'">×</button>
        <img class="modal-content" id="modal-image">
    </div>
</div>
<script>
    function showFullScreen(src) {{
        const modal = document.getElementById('modal');
        const modalImage = document.getElementById('modal-image');
        modalImage.src = src;
        modal.style.display = 'flex';
    }}
</script>
"""

# Preprocess image for consistent detection and size reduction
def preprocess_image(image_array, filename="unknown"):
    try:
        max_size = 800
        h, w = image_array.shape[:2]
        if max(h, w) > max_size:
            scale = max_size / max(h, w)
            image_array = cv2.resize(image_array, (int(w * scale), int(h * scale)), interpolation=cv2.INTER_AREA)
        
        pil_image = Image.fromarray(image_array)
        buffer = io.BytesIO()
        pil_image.save(buffer, format="JPEG", quality=85)
        compressed_image = np.array(Image.open(buffer), dtype=np.uint8)
        
        if compressed_image.shape[-1] == 4:
            compressed_image = cv2.cvtColor(compressed_image, cv2.COLOR_RGBA2RGB)
        elif len(compressed_image.shape) == 2:
            compressed_image = cv2.cvtColor(compressed_image, cv2.COLOR_GRAY2RGB)
            
        return compressed_image
    except Exception as e:
        print(f"Error preprocessing {filename}: {str(e)}")
        return None

# Extract face encodings and locations with HOG
def get_face_encodings_and_locations(image_data, filename="unknown"):
    try:
        image = Image.open(io.BytesIO(image_data))
        if image.mode != 'RGB':
            print(f"Converting {filename} from {image.mode} to RGB")
            image = image.convert('RGB')
        buffer = io.BytesIO()
        image.save(buffer, format="JPEG")
        image_array = np.array(Image.open(buffer), dtype=np.uint8)
        if image_array.dtype != np.uint8 or len(image_array.shape) != 3 or image_array.shape[2] != 3:
            raise ValueError(f"Invalid format for {filename}: Shape {image_array.shape}, Dtype {image_array.dtype}")

        image_array = preprocess_image(image_array, filename)
        if image_array is None:
            return [], [], None
            
        face_locations = face_recognition.face_locations(image_array, model="hog")
        if not face_locations:
            return [], [], image_array
        encodings = face_recognition.face_encodings(image_array, face_locations, num_jitters=3)
        return encodings, face_locations, image_array
    except Exception as e:
        print(f"Error processing {filename}: {str(e)}")
        return [], [], None

# Load images from folder with size check
def load_images(folder_path, progress_bar=None, output=None):
    uploaded_files = {}
    if not os.path.isdir(folder_path):
        print(f"Invalid folder path: {folder_path}")
        return uploaded_files
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    total_files = len(image_files)
    
    max_file_size = 5 * 1024 * 1024
    for i, filename in enumerate(image_files):
        file_path = os.path.join(folder_path, filename)
        file_size = os.path.getsize(file_path)
        
        if file_size > max_file_size:
            print(f"Skipping {filename}: File size ({file_size / 1024 / 1024:.2f}MB) exceeds 5MB limit.")
            continue
            
        try:
            with open(file_path, 'rb') as f:
                image_data = f.read()
            
            if len(image_data) > 1 * 1024 * 1024:
                image = Image.open(io.BytesIO(image_data))
                if image.mode != 'RGB':
                    image = image.convert('RGB')
                buffer = io.BytesIO()
                image.save(buffer, format="JPEG", quality=75)
                image_data = buffer.getvalue()
                print(f"Compressed {filename} to {len(image_data) / 1024 / 1024:.2f}MB")
                
            uploaded_files[filename] = {'content': image_data, 'original_name': filename}
        except Exception as e:
            print(f"Error loading {filename}: {str(e)}")
            continue
            
        if progress_bar:
            progress_bar.value = (i + 1) / total_files
        if output:
            with output:
                clear_output(wait=True)
                print(f"Loading {i + 1}/{total_files}: {filename}")
    return uploaded_files

# Group similar faces
def group_similar_faces(uploaded_files, progress_bar=None, output=None):
    all_encodings = []
    all_filenames = []
    all_image_data = []
    processed_images = 0
    total_files = len(uploaded_files)

    for i, (filename, file_info) in enumerate(uploaded_files.items()):
        image_data = file_info['content']
        encodings, _, _ = get_face_encodings_and_locations(image_data, filename)
        if encodings:
            processed_images += 1
            for encoding in encodings:
                all_encodings.append(encoding)
                all_filenames.append(filename)
                all_image_data.append(image_data)
        if progress_bar:
            progress_bar.value = (i + 1) / total_files
        if output:
            with output:
                clear_output(wait=True)
                print(f"Processing {i + 1}/{total_files}: {filename}")

    if not all_encodings:
        return defaultdict(list), processed_images

    groups = []
    threshold = 0.45
    for encoding, filename, image_data in zip(all_encodings, all_filenames, all_image_data):
        matched = False
        for group in groups:
            if face_recognition.face_distance([group[0]['encoding']], encoding)[0] < threshold:
                group.append({'encoding': encoding, 'filename': filename, 'image_data': image_data})
                matched = True
                break
        if not matched:
            groups.append([{'encoding': encoding, 'filename': filename, 'image_data': image_data}])

    refined_groups = {f"Face {i+1}": group for i, group in enumerate(groups)}
    return refined_groups, processed_images

# Save grouped faces and create ZIP
def save_grouped_faces_and_zip(face_groups, output_folder, uploaded_files):
    zip_files = []
    file_mapping = {}
    os.makedirs(output_folder, exist_ok=True)
    for group_id, faces in face_groups.items():
        group_folder = os.path.join(output_folder, f"{group_id}")
        Path(group_folder).mkdir(exist_ok=True)
        saved_filenames = set()
        for face_data in faces:
            original_name = uploaded_files[face_data['filename']]['original_name']
            if original_name not in saved_filenames:
                unique_name = f"{os.path.splitext(original_name)[0]}_{uuid.uuid4().hex[:8]}{os.path.splitext(original_name)[1]}"
                dest_path = os.path.join(group_folder, unique_name)
                with open(dest_path, 'wb') as f:
                    f.write(face_data['image_data'])
                file_mapping[(group_id, original_name)] = dest_path
                saved_filenames.add(original_name)
        print(f"Saved {len(saved_filenames)} images for {group_id}")

    zip_filename = "all_faces.zip"
    zip_path = os.path.join(output_folder, zip_filename)
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for group_id, faces in face_groups.items():
            saved_filenames = set()
            for face_data in faces:
                original_name = uploaded_files[face_data['filename']]['original_name']
                if original_name not in saved_filenames:
                    image_path = file_mapping.get((group_id, original_name))
                    if image_path and os.path.exists(image_path):
                        zipf.write(image_path, os.path.join(f"{group_id}", os.path.basename(image_path)))
                        saved_filenames.add(original_name)
    zip_files.append(zip_path)
    print(f"ZIP created at: {zip_path}")
    return zip_files, file_mapping

# Display images in separate 3-column grids for each face group with full-screen functionality
def display_grouped_images(face_groups, file_mapping):
    global uploaded_files
    if not file_mapping:
        return "<p>No images to display.</p>"
    
    grouped_images = defaultdict(list)
    for (group_id, original_name), image_path in file_mapping.items():
        grouped_images[group_id].append((original_name, image_path))
    
    image_html = ""
    for group_id, images in sorted(grouped_images.items(), key=lambda x: int(x[0].split()[-1])):
        image_html += f"""
        <div class='face-group'>
            <div class='face-label'>{group_id}</div>
            <div class='image-grid'>
        """
        for original_name, image_path in images:
            try:
                with open(image_path, 'rb') as f:
                    image_data = f.read()
                image_base64 = base64.b64encode(image_data).decode('utf-8')
                image_html += f"""
                <div class='image-card' onclick="showFullScreen('data:image/jpeg;base64,{image_base64}')">
                    <img src='data:image/jpeg;base64,{image_base64}' alt='{group_id}'>
                    <div class='image-overlay'>
                        <div class='group-label'>{group_id}</div>
                        <div class='image-filename'>{os.path.basename(image_path)}</div>
                    </div>
                </div>
                """
            except Exception as e:
                print(f"Error displaying {image_path}: {str(e)}")
        image_html += "</div></div>"
    
    return image_html

# Webcam face recognition and match without share options
def recognize_face_from_webcam(face_groups, uploaded_files, output_folder, file_mapping, output_widget):
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        with output_widget:
            print("Error: Could not open webcam.")
        return None
    while True:
        ret, frame = cap.read()
        if not ret:
            with output_widget:
                print("Error: Could not read frame.")
            break
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(rgb_frame, model="hog")
        if face_locations:
            encodings = face_recognition.face_encodings(rgb_frame, face_locations)
            for encoding in encodings:
                for group_id, faces in face_groups.items():
                    group_encodings = [f['encoding'] for f in faces]
                    distances = face_recognition.face_distance(group_encodings, encoding)
                    if min(distances) < 0.45:
                        cap.release()
                        cv2.destroyAllWindows()
                        with output_widget:
                            clear_output()
                            print(f"Match found: {group_id}")

                            # Create ZIP for matched group
                            zip_path = os.path.join(output_folder, f"{group_id}.zip")
                            with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
                                for (g_id, _), image_path in file_mapping.items():
                                    if g_id == group_id:
                                        zipf.write(image_path, os.path.basename(image_path))
                            relative_path = os.path.relpath(zip_path, start=os.getcwd())

                            # Display result with only download option
                            result_html = (f"Matched face group: {group_id}. "
                                          f"<a href='{relative_path}' download='{group_id}.zip'>Download matched group</a>")
                            with output_widget:
                                display(HTML(result_html))
                        return group_id
        with output_widget:
            clear_output(wait=True)
            print("Scanning for face match...")
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
    with output_widget:
        clear_output()
        print("Webcam session ended.")
    return None

# Main UI
path_input = Text(value='', placeholder='Enter folder path', description='Folder Path:')
process_button = Button(description="Process Images")
webcam_button = Button(description="Recognize via Webcam")
progress_bar = FloatProgress(value=0, min=0, max=1, description='Progress:')
output = Output()

# Global variables
face_groups = None
uploaded_files = None
file_mapping = None
output_folder = "individual_faces_output"

def on_process_button_clicked(b):
    global face_groups, uploaded_files, file_mapping
    with output:
        clear_output()
        start_time = time.time()
        folder_path = path_input.value.strip()
        if not folder_path:
            display(HTML(after_ui_html.format("Please enter a folder path.")))
            return
        
        progress_bar.value = 0
        uploaded_files = load_images(folder_path, progress_bar, output)
        if not uploaded_files:
            display(HTML(after_ui_html.format("No valid image files found in the folder.")))
            return
        
        print(f"Loaded {len(uploaded_files)} files.")
        progress_bar.value = 0
        face_groups, processed_images = group_similar_faces(uploaded_files, progress_bar, output)
        if processed_images == 0:
            display(HTML(after_ui_html.format("No faces detected in the images.")))
            return
        
        print(f"Detected {len(face_groups)} face groups.")
        zip_files, file_mapping = save_grouped_faces_and_zip(face_groups, output_folder, uploaded_files)
        output_text = f"Summary (as of {time.strftime('%Y-%m-%d %H:%M:%S')}):\n"
        output_text += f"Total images processed: {processed_images}\n"
        output_text += f"Number of unique faces detected: {len(face_groups)}\n"
        output_text += f"Execution time: {time.time() - start_time:.2f} seconds\n"
        output_text += "\nFace Groups:\n"
        for group_id, faces in face_groups.items():
            filenames = [uploaded_files[f['filename']]['original_name'] for f in faces]
            output_text += f"{group_id}: {', '.join(set(filenames))}\n"
        
        if zip_files:
            zip_path = zip_files[0]
            relative_path = os.path.relpath(zip_path, start=os.getcwd())
            output_text += f"\n<a href='{relative_path}' download='all_faces.zip'>Download all grouped faces as ZIP</a>"
        
        output_text += "\n\nGrouped Images:\n"
        image_display = display_grouped_images(face_groups, file_mapping)
        display(HTML(after_ui_html.format(output_text + image_display)))

def on_webcam_button_clicked(b):
    global face_groups, uploaded_files, file_mapping
    with output:
        clear_output()
        if not face_groups or not uploaded_files or not file_mapping:
            display(HTML("Please process images first by entering a folder path and clicking 'Process Images'."))
            return
        
        matched_group_id = recognize_face_from_webcam(face_groups, uploaded_files, output_folder, file_mapping, output)
        if not matched_group_id:
            display(HTML("No match found or webcam session ended."))

process_button.on_click(on_process_button_clicked)
webcam_button.on_click(on_webcam_button_clicked)

# Display UI
display(VBox([HBox([path_input]), HBox([process_button, webcam_button]), progress_bar, output]))